<a href="https://colab.research.google.com/github/PWhiddy/ReintegrationTrackingAutodiff/blob/main/ReintegrationTrackingPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autodiff Reintengration Tracking

A pytorch implementation of [Mykhailo Moroz](https://michaelmoroz.github.io) and [Wyatt Flanders](https://https://www.shadertoy.com/user/wyatt) "Reintegration Tracking" particle/fluid algorithm
https://michaelmoroz.github.io/Reintegration-Tracking/

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
torch.__version__

'2.0.0.dev20230226+cu118'

In [3]:
#VideoWriter from Alexander Mordvintsev 
#https://colab.research.google.com/github/znah/notebooks/blob/master/external_colab_snippets.ipynb

import os
import numpy as np
os.environ['FFMPEG_BINARY'] = 'ffmpeg'
import moviepy.editor as mvp
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter

class VideoWriter:
  def __init__(self, filename='_autoplay.mp4', fps=30.0, **kw):
    self.writer = None
    self.params = dict(filename=filename, fps=fps, **kw)

  def add(self, img):
    img = np.asarray(img)
    if self.writer is None:
      h, w = img.shape[:2]
      self.writer = FFMPEG_VideoWriter(size=(w, h), **self.params)
    if img.dtype in [np.float32, np.float64]:
      img = np.uint8(img.clip(0, 1)*255)
    if len(img.shape) == 2:
      img = np.repeat(img[..., None], 3, -1)
    self.writer.write_frame(img)

  def close(self):
    if self.writer:
      self.writer.close()

  def __enter__(self):
    return self

  def __exit__(self, *kw):
    self.close()
    if self.params['filename'] == '_autoplay.mp4':
      self.show()

  def show(self, **kw):
      self.close()
      fn = self.params['filename']
      display(mvp.ipython_display(fn, **kw))

In [4]:
# change to use cpu/gpu
use_gpu = True
dev_name = 'cuda' if use_gpu else 'cpu'
device = torch.device(dev_name)

In [5]:
def create_initial_state(device, batch=1, dim=64):
  st = torch.rand(batch, 5, dim, dim, device=device)
  st[:,2:4,:,:] -= 0.5
  return st

In [6]:
def unfold_kernel_area(st, kernel_size=3):
  kernel_count = kernel_size*kernel_size
  batch_s = st.shape[0]
  cells = st.shape[2]*st.shape[3]
  neighbors = F.unfold(F.pad(st, (1,1,1,1), mode='circular'), (kernel_size, kernel_size) )
  return neighbors.reshape(batch_s, 5, kernel_count, cells) 

In [7]:
def compute_neighbors_overlap(neighbors, diffusion_radius, kernel_size=3):
  # neighbors is [b, 5, 9 (for 3x3), x*y]
  dv = neighbors.device
  cr = torch.arange(0.0, kernel_size, device=dv)-kernel_size//2 # coordinates range
  neighbor_coords = torch.cartesian_prod(cr,cr) # TODO verify these are order which matches unfolded kernels
  # account for cell-relative coordinates
  neighbors[:, 0:2, :, :] += neighbor_coords.t().reshape(1, 2, 9, 1)
  p_l, p_u = neighbors[:, 0:2, :, :] - diffusion_radius, neighbors[:, 0:2, :, :] + diffusion_radius # particle box corners
  over_l, over_u = torch.max(torch.tensor(0.0, device=dv), p_l), torch.min(torch.tensor(1.0, device=dv), p_u)
  over_center = 0.5 * (over_l + over_u)
  size = torch.max(over_u-over_l, torch.tensor(0, device=dv))
  area = (size[:,0,:,:]*size[:,1,:,:])/(4.0*diffusion_radius*diffusion_radius)
  return torch.cat((over_center, area.unsqueeze(1)), 1)

In [8]:
def update_state(st, delta_time, diffusion_radius):
  # integrate particle positions
  st[:,0:2,:,:] += delta_time*st[:,2:4,:,:]
  neighbors = unfold_kernel_area(st)
  overlaps = compute_neighbors_overlap(neighbors, diffusion_radius)
  overlap_mass = overlaps[:, 2:3, :, :] * neighbors[:, 4:5, :, :]
  mass = overlap_mass.sum(2)
  div_mass = mass.detach().clone()
  div_mass[div_mass==0.0] = 1.0
  position = (overlaps[:, 0:2, :, :] * overlap_mass).sum(2) / div_mass
  velocity = (neighbors[:, 2:4, :, :] * overlap_mass).sum(2) / div_mass
  st[:, 0:2, :, :] = position.reshape(st.shape[0], 2, st.shape[2], st.shape[3])
  st[:, 2:4, :, :] = velocity.reshape(st.shape[0], 2, st.shape[2], st.shape[3])
  st[:, 4:5, :, :] = mass.reshape(st.shape[0], 1, st.shape[2], st.shape[3])


In [9]:
cur_state = create_initial_state(device, batch=1, dim=512)
with VideoWriter(fps=60) as vw:
  for i in tqdm(range(5000)):
      if i%100 == 0:
        vw.add(cur_state[0, 4, :, :].cpu())
      update_state(cur_state, torch.tensor(0.6, device=device), torch.tensor(0.5, device=device))

100%|████████████████████████████████████████████████████████████████████████| 5000/5000 [00:02<00:00, 1909.87it/s]


In [10]:
compiled_update = torch.compile(update_state)
# make it run once
compiled_update(create_initial_state(device, batch=1, dim=512), torch.tensor(0.6, device=device), torch.tensor(0.5, device=device))


In [ ]:
cur_state = create_initial_state(device, batch=1, dim=512)
with VideoWriter(fps=60) as vw:
  for i in tqdm(range(50000)):
      if i%1000 == 0:
        vw.add(cur_state[0, 4, :, :].cpu())
      compiled_update(cur_state, torch.tensor(0.6, device=device), torch.tensor(0.5, device=device))

 12%|████████▌                                                              | 6010/50000 [00:01<00:07, 5598.23it/s]

In [36]:
# broke colab gpu
!nvidia-smi

Sun Jan 10 03:13:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    36W /  70W |   1289MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
# epic gaming device of ultimate enlightenment 
!nvidia-smi

Sun Jul 16 00:31:48 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:01:00.0  On |                  Off |
| 32%   49C    P8              38W / 450W |   1408MiB / 24564MiB |      4%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--